#### Obtain ECHR data from JSONL file

In [1]:
import pandas as pd

def articles_of_interest(a):
    if(a==0):
        return False
    articles = ['0','P1-1','5','3','13','8','10','2']
    return a[0] in articles

def obtain_ECHR_data(file, as_disc = False, n_sample = 300):
    jsonObj = pd.read_json(path_or_buf='./echr_dataset/'+file, lines=True)
    data_raw = jsonObj[['facts','violated_articles']]
    #data_raw = data_raw.head(n=5)#debugging
    data = data_raw.copy()
    data['information'] = data_raw['facts'].apply(lambda x: '. '.join([e[4:] for e in x]))
    data['guilty'] = data_raw['violated_articles'].apply(lambda x: ['6','a','b'] if x==['6'] else x)
    data['guilty'] = data['guilty'].apply(lambda x: [e for e in x if e!='6'])
    data['guilty'] = data['guilty'].apply(lambda x: ['0'] if len(x)==0 else x)
    data['guilty'] = data['guilty'].apply(lambda x: 0 if len(x)>1 else x)
    data['guilty'] = data['guilty'].apply(lambda x: 0 if not articles_of_interest(x) else x)
    data = data.drop(data[data.guilty == 0].index)
    data['guilty'] = data['guilty'].apply(lambda x: 1 if x[0]=='P1-1' else int(x[0]))
    if(not as_disc):
        data['guilty'] = data['guilty'].apply(lambda x: str(x))
        data = data.groupby('guilty').sample(n=n_sample, replace=True)
        return data['information'].to_numpy(), pd.get_dummies(data['guilty']).to_numpy()
    else:
        data = data.groupby('guilty').sample(n=n_sample, replace=True)
        return data[['information','guilty']].to_dict('records')

In [2]:
#train_examples, train_labels = obtain_ECHR_data('train.jsonl')
#train_labels

In [3]:
x_train, y_train = obtain_ECHR_data('train.jsonl')
x_val, y_val = obtain_ECHR_data('dev.jsonl')
#test_data = obtain_ECHR_data('test.jsonl', as_disc = True)

#### Obtain model from Google

In [6]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

In [7]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(
    model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True
)
#hub_layer(x_train[:3])

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(8))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
Total params: 400492 (1.53 MB)
Trainable params: 400492 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Train Model

In [9]:
model.compile(
    optimizer="adam",
    loss=tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [11]:
history = model.fit(
    x_train,
    y_train,
    epochs=40,
    batch_size=512,
    validation_data=(x_val, y_val),
    verbose=1,
)

Epoch 1/40
5/5 [==============================] - 1s 326ms/step - loss: 1.2602 - accuracy: 0.6029 - val_loss: 1.7663 - val_accuracy: 0.4521
Epoch 2/40
5/5 [==============================] - 1s 317ms/step - loss: 1.2427 - accuracy: 0.6108 - val_loss: 1.7561 - val_accuracy: 0.4588
Epoch 3/40
5/5 [==============================] - 1s 313ms/step - loss: 1.2261 - accuracy: 0.6179 - val_loss: 1.7466 - val_accuracy: 0.4604
Epoch 4/40
5/5 [==============================] - 1s 310ms/step - loss: 1.2095 - accuracy: 0.6233 - val_loss: 1.7361 - val_accuracy: 0.4692
Epoch 5/40
5/5 [==============================] - 1s 311ms/step - loss: 1.1933 - accuracy: 0.6271 - val_loss: 1.7287 - val_accuracy: 0.4833
Epoch 6/40
5/5 [==============================] - 1s 310ms/step - loss: 1.1775 - accuracy: 0.6329 - val_loss: 1.7181 - val_accuracy: 0.4929
Epoch 7/40
5/5 [==============================] - 1s 321ms/step - loss: 1.1621 - accuracy: 0.6388 - val_loss: 1.7104 - val_accuracy: 0.5017
Epoch 8/40
5/5 [====

#### Create wrapper for model

In [12]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        return logits

/home/ronaldo/anaconda3/envs/text-attack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
CustomTensorFlowModelWrapper(model)(["He is guilty", "She is innocent"])

tensor([[0.4312, 0.4088, 0.4630, 0.4558, 0.3266, 0.4432, 0.5016, 0.4526],
        [0.4967, 0.3802, 0.5205, 0.4542, 0.3941, 0.5621, 0.5388, 0.5446]])

In [14]:
model_wrapper = CustomTensorFlowModelWrapper(model)

In [15]:
import textattack
from datasets import Dataset

test_data = obtain_ECHR_data('test.jsonl', as_disc = True, n_sample = 5)
dataset = Dataset.from_list(test_data)
dataset = textattack.datasets.HuggingFaceDataset(dataset, dataset_columns=(['information'], 'guilty'))

In [16]:
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker

attack = PWWSRen2019.build(model_wrapper)

[nltk_data] Downloading package omw-1.4 to /home/ronaldo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [16]:
attacker = Attacker(attack, dataset)

#### Attack!

In [17]:
results = attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  20%|██        | 2/10 [00:00<00:00, 137.38it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[3 (14%)]] --> [[[SKIPPED]]]

The applicant was born in 1973 and lives in Oslo, Norway.. On 26 April 2000 criminal proceedings were brought against the applicant and one other person in connection with a traffic accident which had resulted in death of a child.. On 25 February 2003 the Court of First Instance in Podgorica convicted the applicant for endangering public traffic and sentenced him and his co-accused to one year and six months’ imprisonment.. On an unspecified date in 2006 the High Court in Podgorica quashed this judgment and remitted case back to the Court of First Instance.. On 18 April 2007 the Court of First Instance adopted a new judgment and again convicted the applicant and his co-accused. But the court reduced the sentence to one year and four months’ imprisonment.. On 13 November 2009 the High Court further reduced the sentence of the applicant’s co-accused, but uph

[Succeeded / Failed / Skipped / Total] 0 / 0 / 2 / 2:  30%|███       | 3/10 [01:34<03:40, 31.52s/it] 

--------------------------------------------- Result 3 ---------------------------------------------


...

--------------------------------------------- Result 7 ---------------------------------------------
[[0 (18%)]] --> [[[FAILED]]]

The applicants, who belong to the Roma ethnic group, were born in 1970 and 1957 respectively and live in Mănăstirea (Călărași).. At the relevant time they were living together in a de facto partnership and were the parents of seven children:
–  E. and E.S., a boy and a girl, twins, born on 27 March 2004;
–  T., a boy, born on 7 March 2005;
–  A.-M., a girl, born on 31 March 2006;
–  S., a girl, born on 15 June 2007;
–  E.L., a girl, born on 29 June 2008;
–  I., a boy, born on 8 October 2009.. The second applicant has been classified as suffering from second‑degree permanent disability on account of psychological disorders.. On 3 October 2013 the applicants married.. In August 2010 the first applicant sent a complaint to the Romanian president, claiming that her father had subjected her to abuse; she stated that he had raped her and threatened her children. 

[Succeeded / Failed / Skipped / Total] 4 / 1 / 4 / 9:  90%|█████████ | 9/10 [1:03:38<07:04, 424.32s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[0 (17%)]] --> [[7 (17%)]]

The applicant was born in 1925 and at the time of the introduction of the application lived in Dublin.. On 20 December 2012 X. (around 55 years old) instituted an action before the Civil Court (Family Section) requesting the court to declare the applicant to be her biological father and to order this to be reflected on her birth certificate.. On 11 February 2013 the Director of the Public Registry (also defendant in the proceedings) requested the court to order that the parties submit themselves to the genetic tests as provided by Article 100A of the Civil Code, and for the applicant to submit his details, which would be required to [[make]] the relevant changes to [[X]].’s birth certificate, if the court had to find in [[X]].’s favour..  On 11 February 2013 the applicant filed written submissions in reply, denying that he had been involved in the applicant’

[Succeeded / Failed / Skipped / Total] 5 / 1 / 4 / 10: 100%|██████████| 10/10 [1:11:00<00:00, 426.01s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[0 (20%)]] --> [[7 (16%)]]

The [[first]] applicant was born in 1983. She lives in Vilnius and has three children. The [[first]] time she gave birth was in 2009 in a public hospital in Vilnius. According to the applicant, the doctors therein were rude and rebuked her for her wish to give birth at home, and she felt humiliated and for a long time afterwards could not discuss her [[experience]] of giving birth at the hospital without tears coming to her eyes. For those reasons, in 2011 she gave birth to her [[second]] child at home, with the assistance of an unlicensed doula (pribuvėja), J.I.Š. That birth passed without complications and without the need for medical intervention.. After falling pregnant for a third time, and with no possibility of giving birth at home owing to the fact that criminal charges had been brought in respect of the above-mentioned doula, J.I.Š. (see paragraphs

### Let's try another attack

In [29]:

from textattack.attack_recipes import Pruthi2019
attack2 = Pruthi2019.build(model_wrapper)
attacker2 = Attacker(attack2, dataset)

textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [30]:
results2 = attacker2.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 





  0%|          | 0/10 [1:19:40<?, ?it/s]A


 10%|█         | 1/10 [43:57<6:35:36, 2637.42s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [43:57<6:35:36, 2637.42s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|██        | 2/10 [43:57<2:55:49, 1318.72s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 2 / 3:  30%|███       | 3/10 [43:57<1:42:34, 879.15s/it] 

[Succeeded / Failed / Skipped / Total] 0 / 1 / 3 / 4:  40%|████      | 4/10 [43:57<1:05:56, 659.36s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 4 / 5:  50%|█████     | 5/10 [43:57<43:57, 527.49s/it]  

[Succeeded / Failed / Skipped / Total] 0 / 1 / 5 / 6:  60%|██████    | 6/10 [43:57<29:18, 439.58s/it]

[Succeeded / Failed / Skipped / Total] 0 / 1 / 6 / 7:  70%|███████   | 7/10 [43:57<18:50, 376.78s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (13%)]] --> [[[FAILED]]]

The applicant was born in Nigeria in 1975 and lives in Zölling.. In Nigeria, the applicant completed eleven years of schooling and subsequently held various jobs in Lagos. After leaving the country, he lived in Spain from 1994 to 1997, living at first on social security and later working as a day-labourer on several farms. In 1997 he arrived in Germany, where he applied for asylum under a different identity. His application was finally rejected in July 1998. He left Germany for Italy with a German national with whom he had started a relationship. In January 1999 he moved on to Spain, where he worked as a cook and waiter in various restaurants.. In October 2000 the applicant re-entered Germany. On 21 October 2000 his daughter, from his above-mentioned relationship, was born. The applicant’s daughter is a German national.. In 2000 and 2001 the applicant and t



[Succeeded / Failed / Skipped / Total] 0 / 1 / 6 / 7:  80%|████████  | 8/10 [48:25<12:06, 363.23s/it]

[Succeeded / Failed / Skipped / Total] 0 / 2 / 6 / 8:  80%|████████  | 8/10 [48:25<12:06, 363.23s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (18%)]] --> [[[FAILED]]]

The first, second and third applicants were born in 1971, 1947 and 1954 respectively, and live in Budapest.. The second and third applicants, as well as the legal predecessors of the first and third applicant (Ms Róbertné Hidas and Ms Miklósné Kecskés) and other persons were co-owners of a plot of land of 4,847 sq. m (land registry title no. 223591) in District XXII of Budapest. They also co‑owned a neighbouring plot of land of 423 sq. m (land registry title no. 223592).. On 19 January 1998 the Budapest District XXII Mayor’s Office (hereafter “the local authority”) ordered the partition of plot no. 223591 into two plots measuring 1,519 sq. m and 3,328 sq. m. In the same decision, it requisitioned (lejegyzés) the plot of 1,519 sq. m for the purpose of constructing a local public road and awarded compensation of 5,511,000 Hungarian forints (HUF – approximatel



[Succeeded / Failed / Skipped / Total] 0 / 2 / 6 / 8:  90%|█████████ | 9/10 [48:56<05:26, 326.26s/it]

[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9:  90%|█████████ | 9/10 [48:56<05:26, 326.26s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (17%)]] --> [[[FAILED]]]

The list of applicants is set out in the appended tables.. The applicants were employees of a municipal education institution.. On 13 June 2001 they brought proceedings before the Kolomna Town Court of the Moscow Region (“the Town Court”) against their employer and the town administration seeking recovery of unpaid wages and taxes to various social funds.. On 13 July 2004 the Kolomna Town Court issued a first-instance judgment on the merits of the case. On 22 September 2004 the Moscow Regional Court quashed the judgment on appeal and remitted the case for a fresh examination.. On 7 July 2005 the Town Court granted the applicants’ claims awarding each of them a certain amount against their employer (see Appendix I). The judgment was upheld on appeal by the Moscow Regional Court on 28 September 2005..  Between 26 June 2001 and 7 July 2005 the court hearings w



[Succeeded / Failed / Skipped / Total] 0 / 3 / 6 / 9: 100%|██████████| 10/10 [50:40<00:00, 304.03s/it]

[Succeeded / Failed / Skipped / Total] 0 / 4 / 6 / 10: 100%|██████████| 10/10 [50:40<00:00, 304.03s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[1 (20%)]] --> [[[FAILED]]]

The applicants are owners of residential buildings or apartments which were subject to the rent-control scheme. Under the relevant legislation they were obliged to let their flats to tenants while charging no more than the maximum amount of rent fixed by the State. The legislation precluded them from unilaterally terminating the leases or selling the flats in question to anyone other than the respective tenants. The particulars of the flats affected by the rent control are set out in Appendices 10 ‑ 17 (columns A ‑ F).. The situation of the applicants is structurally and contextually the same as that of the applicants in Bittó and Others v. Slovakia (no. 30255/09, 28 January 2014 (merits) and 7 July 2015 (just satisfaction)), and subsequently decided cases concerning the rent-control scheme in Slovakia (see Krahulec v. Slovakia, no. 19294/07; Bukovčanová a


/home/ronaldo/anaconda3/envs/text-attack/lib/python3.11/site-packages/textattack/metrics/attack_metrics/words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
/home/ronaldo/anaconda3/envs/text-attack/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### Let's try another attack

In [26]:
from textattack.attack_recipes import DeepWordBugGao2018
attack3 = DeepWordBugGao2018.build(model_wrapper)
attacker3 = Attacker(attack3, dataset)

textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [28]:
#results3 = attacker3.attack_dataset() #time-out

### Let's try another attack

In [22]:
from textattack.attack_recipes import TextBuggerLi2018
attack4 = TextBuggerLi2018.build(model_wrapper)
attacker4 = Attacker(attack4, dataset)

textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [23]:
results4 = attacker4.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [02:09<19:26, 129.65s/it]

--------------------------------------------- Result 1 ---------------------------------------------



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [02:09<19:29, 129.92s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  20%|██        | 2/10 [02:09<08:39, 64.97s/it] 
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [02:09<08:39, 64.97s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 2 / 3:  30%|███       | 3/10 [02:09<05:03, 43.31s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 3 / 4:  40%|████      | 4/10 [02:09<03:14, 32.49s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 4 / 5:  50%|█████     | 5/10 [02:09<02:09, 25.99s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 5 / 6:  60%|██████    | 6/10 [02:09<01:26, 21.66s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 6 / 7:  70%|███████   | 7/10 [02:09<00:55, 18.57s/it]

[[0 (13%)]] --> [[6 (13%)]]

The applicant was born in [[Nigeria]] in 1975 and lives in Zölling.. In [[Nigeria]], the applicant completed eleven years of schooling and subsequently held various jobs in Lagos. After leaving the country, he lived in [[Spain]] from 1994 to 1997, living at first on social security and later working as a day-labourer on several farms. In 1997 he arrived in [[Germany]], where he applied for asylum under a different [[identity]]. His application was finally rejected in July 1998. He left [[Germany]] for Italy with a [[German]] national with whom he had started a relationship. In January 1999 he moved on to Spain, where he worked as a cook and waiter in various restaurants.. In October 2000 the applicant re-entered [[Germany]]. On 21 October 2000 his daughter, from his [[above-mentioned]] relationship, was born. The applicant’s daughter is a German national.. In 2000 and 2001 the applicant and the child’s mother lived together. From the outset they had, and co

...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (18%)]] --> [[0 (13%)]]

The first, second and third applicants were born in 1971, 1947 and [[1954]] respectively, and live in Budapest.. The second and third applicants, as well as the legal predecessors of the first and third applicant (Ms Róbertné Hidas and Ms Miklósné Kecskés) and other persons were [[co-owners]] of a [[plot]] of [[land]] of [[4]],847 sq. m ([[land]] registry title no. 223591) in District XXII of Budapest. They also co‑[[owned]] a neighbouring [[plot]] of [[land]] of 423 sq. m ([[land]] registry title no. 223592).. [[On]] 19 January 1998 the Budapest District XXII Mayor’s Office (hereafter “the local [[authority]]”) ordered the partition of [[plot]] no. 223591 into two plots measuring 1,519 sq. m and 3,328 sq. m. [[In]] the same decision, it requisitioned (lejegyzés) the [[plot]] of 1,519 sq. m for the purpose of constructing a local public road and [[awarded]] 


[Succeeded / Failed / Skipped / Total] 2 / 0 / 6 / 8:  90%|█████████ | 9/10 [02:25<00:16, 16.20s/it]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 6 / 9:  90%|█████████ | 9/10 [02:25<00:16, 16.20s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (17%)]] --> [[3 (15%)]]

The list of applicants is set out in the appended tables.. The applicants were employees of a municipal education institution.. On 13 June 2001 they brought proceedings before the Kolomna Town Court of the Moscow [[Region]] (“the Town Court”) against their employer and the town administration seeking recovery of unpaid wages and taxes to various [[social]] [[funds]].. On 13 July 2004 the Kolomna Town Court issued a first-instance judgment on the merits of the case. On 22 September 2004 the Moscow Regional Court quashed the judgment on appeal and remitted the case for a fresh examination.. On 7 July 2005 the Town Court granted the applicants’ claims awarding each of them a certain amount against their employer (see Appendix I). The judgment was upheld on appeal by the Moscow Regional Court on 28 September 2005..  [[Between]] 26 June 2001 and 7 July 2005 the c


[Succeeded / Failed / Skipped / Total] 3 / 0 / 6 / 9: 100%|██████████| 10/10 [02:31<00:00, 15.11s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 6 / 10: 100%|██████████| 10/10 [02:31<00:00, 15.11s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[1 (20%)]] --> [[5 (15%)]]

The applicants are owners of residential buildings or apartments which were subject to the rent-control scheme. Under the relevant legislation they were obliged to let their flats to [[tenants]] while charging no more than the maximum amount of [[rent]] fixed by the State. The legislation precluded them from unilaterally terminating the leases or selling the flats in question to anyone other than the respective [[tenants]]. The particulars of the flats affected by the rent control are set out in Appendices 10 ‑ 17 (columns A ‑ F).. The situation of the applicants is structurally and contextually the same as that of the applicants in Bittó and Others [[v]]. Slovakia (no. 30255/09, 28 January 2014 (merits) and 7 July 2015 (just satisfaction)), and subsequently decided cases concerning the rent-control scheme in Slovakia (see Krahulec [[v]]. Slovakia, no. 1929

### Let's try another attack

In [24]:
from textattack.attack_recipes import InputReductionFeng2018
attack5 = InputReductionFeng2018.build(model_wrapper)
attacker5 = Attacker(attack5, dataset)

textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.


In [25]:
results5 = attacker5.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  InputReduction(
    (maximizable):  True
  )
  (transformation):  WordDeletion
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 




 10%|█         | 1/10 [01:53<17:04, 113.78s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [01:53<17:04, 113.85s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [01:53<07:35, 56.93s/it] 
[Succeeded / Failed / Skipped / Total] 1 / 0 / 2 / 3:  30%|███       | 3/10 [01:53<04:25, 37.96s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 3 / 4:  40%|████      | 4/10 [01:53<02:50, 28.47s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 3 / 4:  50%|█████     | 5/10 [01:53<01:53, 22.78s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 4 / 5:  50%|█████     | 5/10 [01:53<01:53, 22.78s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 5 / 6:  60%|██████    | 6/10 [01:53<01:15, 18.98s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 6 / 7:  70%|███████   | 7/10 [01:53<00:48, 16.27s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (13%)]] --> [[0 (13%)]]

[[The]] [[applicant]] was [[born]] in [[Nigeria]] in [[1975]] and [[lives]] [[in]] Zölling.. [[In]] Nigeria, [[the]] applicant [[completed]] [[eleven]] [[years]] [[of]] schooling [[and]] subsequently [[held]] [[various]] [[jobs]] in [[Lagos]]. [[After]] [[leaving]] the country, [[he]] [[lived]] [[in]] [[Spain]] [[from]] 1994 to 1997, living [[at]] [[first]] on [[social]] [[security]] [[and]] [[later]] [[working]] as [[a]] [[day-labourer]] [[on]] several [[farms]]. [[In]] 1997 [[he]] arrived [[in]] [[Germany]], where he applied [[for]] asylum [[under]] a [[different]] identity. [[His]] [[application]] [[was]] [[finally]] [[rejected]] in [[July]] [[1998]]. [[He]] left Germany [[for]] Italy with [[a]] [[German]] [[national]] with [[whom]] [[he]] [[had]] started a [[relationship]]. [[In]] January 1999 [[he]] [[moved]] [[on]] to Spain, where [[he]] worked [[as]] 


[Succeeded / Failed / Skipped / Total] 2 / 0 / 6 / 8:  80%|████████  | 8/10 [02:04<00:31, 15.62s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (18%)]] --> [[1 (19%)]]

[[The]] [[first]], [[second]] and third [[applicants]] [[were]] [[born]] in [[1971]], 1947 and 1954 [[respectively]], and [[live]] in [[Budapest]].. The [[second]] [[and]] [[third]] [[applicants]], as [[well]] [[as]] the [[legal]] [[predecessors]] [[of]] [[the]] [[first]] [[and]] [[third]] applicant ([[Ms]] [[Róbertné]] [[Hidas]] [[and]] Ms [[Miklósné]] Kecskés) and other [[persons]] were [[co-owners]] [[of]] [[a]] plot of land [[of]] [[4]],[[847]] [[sq]]. [[m]] ([[land]] registry [[title]] [[no]]. 223591) in District XXII [[of]] [[Budapest]]. [[They]] [[also]] co‑[[owned]] [[a]] neighbouring [[plot]] [[of]] land [[of]] [[423]] sq. [[m]] (land registry title no. [[223592]]).. [[On]] [[19]] January 1998 [[the]] [[Budapest]] District [[XXII]] Mayor’[[s]] Office (hereafter “the local [[authority]]”) ordered [[the]] partition [[of]] plot [[no]]. 223591 [[into]] 


[Succeeded / Failed / Skipped / Total] 2 / 0 / 6 / 8:  90%|█████████ | 9/10 [02:06<00:14, 14.09s/it]
[Succeeded / Failed / Skipped / Total] 3 / 0 / 6 / 9:  90%|█████████ | 9/10 [02:06<00:14, 14.09s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[1 (17%)]] --> [[1 (18%)]]

[[The]] [[list]] of [[applicants]] is [[set]] [[out]] [[in]] the appended [[tables]].. [[The]] applicants [[were]] [[employees]] [[of]] [[a]] [[municipal]] education [[institution]].. On [[13]] [[June]] [[2001]] [[they]] brought [[proceedings]] before the [[Kolomna]] [[Town]] Court [[of]] the Moscow Region (“[[the]] [[Town]] [[Court]]”) against their employer [[and]] the [[town]] administration [[seeking]] [[recovery]] [[of]] unpaid [[wages]] and taxes to [[various]] [[social]] funds.. [[On]] [[13]] [[July]] [[2004]] the Kolomna [[Town]] Court issued [[a]] [[first-instance]] [[judgment]] on the [[merits]] [[of]] [[the]] case. [[On]] [[22]] September 2004 [[the]] [[Moscow]] Regional Court [[quashed]] the [[judgment]] [[on]] [[appeal]] [[and]] remitted [[the]] [[case]] [[for]] [[a]] [[fresh]] examination.. [[On]] 7 July [[2005]] [[the]] Town [[Court]] granted 


[Succeeded / Failed / Skipped / Total] 3 / 0 / 6 / 9: 100%|██████████| 10/10 [02:12<00:00, 13.21s/it]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 6 / 10: 100%|██████████| 10/10 [02:12<00:00, 13.22s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[1 (20%)]] --> [[1 (19%)]]

[[The]] [[applicants]] [[are]] owners of residential [[buildings]] [[or]] [[apartments]] [[which]] [[were]] [[subject]] to the [[rent-control]] [[scheme]]. Under the [[relevant]] legislation [[they]] [[were]] obliged to [[let]] their [[flats]] to tenants [[while]] [[charging]] [[no]] [[more]] [[than]] the [[maximum]] [[amount]] [[of]] [[rent]] [[fixed]] [[by]] the State. [[The]] legislation [[precluded]] them [[from]] [[unilaterally]] terminating the leases [[or]] selling [[the]] [[flats]] in [[question]] [[to]] [[anyone]] other [[than]] [[the]] respective [[tenants]]. [[The]] particulars of the [[flats]] [[affected]] [[by]] [[the]] [[rent]] [[control]] are [[set]] [[out]] [[in]] [[Appendices]] 10 ‑ [[17]] (columns [[A]] ‑ [[F]]).. [[The]] [[situation]] [[of]] [[the]] [[applicants]] is [[structurally]] [[and]] [[contextually]] [[the]] same [[as]] that [[of]